# Laboratorio 1

###Inteligencia de Negocios

###Integrantes:
    Juan Diego Cardona 201819447
    Nicolas Ortega XXXXX
    Camila Teran XXXX

# Análisis Exploratorio y Perfilamiento de Datos

Realizar un análisis exploratorio y perfilamiento de datos que le permita entenderlos, al igual que identificar el nivel de calidad de los datos y las tareas de transformación que se requieren para construir modelos de clasificación utilizando técnicas como KNN y árboles de decisión entre otras.
Adicionalmente, la entidad desea incluir como entregable, tableros de control para apoyar tareas de entendimiento de datos y nivel de calidad de los mismos, para lo cual sugiere mostrar estadisticas tales como cantidad de columnas, cantidad de datos, valores nulos, promedio de cada variable entre otros posibles datos que puedan proveer información útil a SaludAlpes.

### 1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [5]:
# Librerías para manejo de datos
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib as mplt
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 
from sklearn import tree

### 2. Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [7]:
sns.__version__
mplt.__version__

'3.3.2'

In [9]:
# Se cargan los datos. 
datos=pd.read_csv('datos.csv', sep=';', encoding = 'utf-8', index_col=0)
datos

,Pregnancies,Glucose,City,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Hair color,,,,,,,,,,
Red,6.0,148.0,New York,72.0,35.0,0.0,336,627.0,50,1.0
Black,1.0,85.0,New York,66.0,29.0,0.0,266,351.0,31,0.0
Red,8.0,183.0,New York,64.0,0.0,0.0,233,672.0,32,1.0
Black,1.0,89.0,New York,66.0,23.0,94.0,281,167.0,21,0.0
Black,0.0,137.0,New York,40.0,35.0,168.0,431,2288.0,33,1.0
...,...,...,...,...,...,...,...,...,...,...
Black,10.0,101.0,New York,76.0,48.0,180.0,329,171.0,63,0.0
Black,2.0,122.0,New York,70.0,27.0,0.0,368,34.0,27,0.0
Red,5.0,121.0,New York,72.0,23.0,112.0,262,245.0,30,0.0


In [10]:
# Cantidad de datos y número de variables
datos.shape

(768, 10)

In [12]:
# Mostrar los datos
datos.head()

,Pregnancies,Glucose,City,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Hair color,,,,,,,,,,
Red,6.0,148.0,New York,72.0,35.0,0.0,336,627.0,50,1.0
Black,1.0,85.0,New York,66.0,29.0,0.0,266,351.0,31,0.0
Red,8.0,183.0,New York,64.0,0.0,0.0,233,672.0,32,1.0
Black,1.0,89.0,New York,66.0,23.0,94.0,281,167.0,21,0.0
Black,0.0,137.0,New York,40.0,35.0,168.0,431,2288.0,33,1.0


In [19]:
# Podemos ver los tipos de todas la variables.
datos.dtypes

Pregnancies                 object
Glucose                     object
City                        object
BloodPressure               object
SkinThickness               object
Insulin                     object
BMI                          int64
DiabetesPedigreeFunction    object
Age                          int64
Outcome                     object
dtype: object

In [16]:
# Y hacer una descripción de los datos
datos.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,767.000000,767.000000,767.000000,767.000000,767.000000,768.000000,767.000000,768.000000,767.000000
mean,3.839635,120.921773,69.096480,20.563233,79.903520,289.796875,432.395046,38.011719,0.349413
std,3.368429,31.984561,19.366833,15.945349,115.283105,116.757554,336.144934,117.825600,0.477096
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,251.750000,205.500000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,309.000000,337.000000,29.000000,0.000000
75%,6.000000,140.500000,80.000000,32.000000,127.500000,359.000000,592.000000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,671.000000,2329.000000,3256.000000,1.000000


In [17]:
# Se observa que hay ausencias, en particular en la variable "decibel_range". Veamos cuantas, para todas las variables:
datos.isnull().sum()

Pregnancies                 1
Glucose                     1
City                        1
BloodPressure               1
SkinThickness               1
Insulin                     1
BMI                         0
DiabetesPedigreeFunction    1
Age                         0
Outcome                     1
dtype: int64

# Justificación de preprocesamiento y limpieza de Datos

Primero vamos a ejecutar los pasos de limpieza de los datos, relacionados el tratamiento de ausencias y registros duplicados.

In [101]:
# Es recomendable que todos los pasos de limpieza y preparación se realicen sobre otro archivo.
datoslimpios = datos
datoslimpios


,Pregnancies,Glucose,City,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Hair color,,,,,,,,,,
Red,6.0,148.0,New York,72.0,35.0,0.0,336,627.0,50,1.0
Black,1.0,85.0,New York,66.0,29.0,0.0,266,351.0,31,0.0
Red,8.0,183.0,New York,64.0,0.0,0.0,233,672.0,32,1.0
Black,1.0,89.0,New York,66.0,23.0,94.0,281,167.0,21,0.0
Black,0.0,137.0,New York,40.0,35.0,168.0,431,2288.0,33,1.0
...,...,...,...,...,...,...,...,...,...,...
Black,10.0,101.0,New York,76.0,48.0,180.0,329,171.0,63,0.0
Black,2.0,122.0,New York,70.0,27.0,0.0,368,34.0,27,0.0
Red,5.0,121.0,New York,72.0,23.0,112.0,262,245.0,30,0.0


In [102]:
# Eliminación registros con ausencias
#¿Porqué realizar este paso?
datoslimpios = datoslimpios.dropna()

En este punto es importante reflexionar en otra alternativa para no perder registros por celdas con valores nulos. ¿Cuándo tendrá sentido utilizar una estrategia distinta a eliminar los registros?

In [103]:
datoslimpios.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.839635,120.921773,69.096480,20.563233,79.903520,289.670143,432.395046,38.006519,0.349413
std,3.368429,31.984561,19.366833,15.945349,115.283105,116.780873,336.144934,117.902397,0.477096
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,251.500000,205.500000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,309.000000,337.000000,29.000000,0.000000
75%,6.000000,140.500000,80.000000,32.000000,127.500000,359.000000,592.000000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,671.000000,2329.000000,3256.000000,1.000000


In [104]:
# Cantidad de datos y número de variables
datoslimpios.shape

(767, 10)

In [96]:
#Restricciones
#Edad menor a 100 y mayor a 21




Es muy importante analizar el objetivo del modelo analítico a construir antes de iniciar a tomar decisiones. En este caso, por ejemplo, se debe aplicar la regla dada por el negocio, sobre qué significa una canción popular y que no, para generar la nueva variable que será la clase (variable objetivo).

Esta nueva columna la derivaremos a partir de la variable "popularity" utilizando un umbral que nos permita determinar cuándo una canción es popular. Si el valor en esta variable es mayor que el umbral entonces ese registro se clasifica como "popular" y será indicado con un "1", de lo contrario será "no popular y esta categoría la representaremos con un "0".

In [ ]:
# Podemos ver los estadísticos de la variable "popularity"
df_tracks_t['popularity'].describe()

In [ ]:
#El observar las variables de forma gráfica puede ayudar al negocio a decidir el punto a partir del cual se tienen canciones populares.
fig=plt.figure(figsize=(12,8))
ax = sns.boxplot("popularity", data=df_tracks_t, orient="v")

In [ ]:
# Ahora definimos la función que nos va a permitir construir nuestra clase.
def label_popularity (row):
    if row['popularity'] > 33 :
        return 1
    return 0
df_tracks_t['popularity_label']=df_tracks_t.apply (lambda row: label_popularity(row), axis=1)

In [ ]:
df_tracks_t.head()

In [ ]:
# Cantidad de datos de cada clase
pd.value_counts(df_tracks_t['popularity_label'])

In [ ]:
ax = sns.countplot(x='popularity_label', data=df_tracks_t)

In [ ]:
# Eliminaremos las variables que consideramos no son útiles para la tarea que queremos resolver, como los identificadores.
df_tracks_t = df_tracks_t.drop(['id', 'id_artists'], axis=1)

Un aspecto muy importante para tener en cuenta son los requerimientos de entrada de los algoritmos de aprendizaje implementados en las librerías utilizadas. En particular, scikit-learn requiere que todos los atributos sean numéricos y que la variable objetivo, en una tarea de clasificación, esté codificada con números.

In [ ]:
# Vamos a seleccionar de nuestro conjunto solo los atributos numéricos.
number_cols = df_tracks_t.dtypes[(df_tracks_t.dtypes == np.int64) | (df_tracks_t.dtypes == np.float64)].index 
number_cols = df_tracks_t.select_dtypes(include = ['int64','float']).columns
number_cols

In [ ]:
df_tracks_m = df_tracks_t[number_cols]

In [ ]:
# Podemos ver como quedaron los datos
df_tracks_m.head()

In [ ]:
# Revisión del total de registros
df_tracks_m.shape

# Clasificadores:

## K-Nearest-neighbours

En este punto, deben justificar las decisiones más importantes asociadas al proceso, tales como los criterios utilizados para la selección de hiperparámetros y las modificaciones a los datos para construir cada clasificador.

## Arboles de Decisión

En este punto, deben justificar las decisiones más importantes asociadas al proceso, tales como los criterios utilizados para la selección de hiperparámetros y las modificaciones a los datos para construir cada clasificador.

## Clasificador de Libre Elección

En este punto, deben justificar las decisiones más importantes asociadas al proceso, tales como los criterios utilizados para la selección de hiperparámetros y las modificaciones a los datos para construir cada clasificador.

# Análisis de Resultados Obtenidos y Utilidad de los Modelos

# Comparación de modelos obtenidos y Recomendación